<a href="https://colab.research.google.com/github/dubuel/MLPytorch/blob/main/dubuel./backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 수치 미분을 이용한 심층 신경망 학습

In [6]:
import time
import numpy as np

## 유틸리티 함수

In [7]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid 구현

In [8]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1

    def __call__(self, x):
        self.last_o = 1/ (1.0 + np.exp(-x))
        return self.last_o

    def grad(self):
        return self.last_o*(1-self.last_o)

## Mean Squared Error 구현

In [9]:
class MeanSquaredError:
    def __init__(self):
        # gradient & h - y initialize
        self.dh = 1
        self.last_diff = 1

    def __call__(self, h, y):
        self.last_diff = h - y
        return 1 / 2 *np.mean(np.square(self.last_diff))

    def grad(self):
        return self.last_diff

## 뉴런 구현

In [15]:
class Neuron:
    def __init__(self, W, b, a_obj):
        self.W = W
        self.b = b
        self.a = a_obj()

        #gradient store
        self.dw =  np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W))

        self.last_x = np.zeros((self.W.shape[0]))
        self.last_h = np.zeros((self.W.shape[1]))

    def __call__(self, x): #forward inference
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self): #dy/dh = W (y = Wx + b, x == h )
        return self.W * self.a.grad()

    def grad_W(self, dh): #dy/dW
        grad= np.ones_like(self.W)
        grad_a = self.a.grad()
        for j in range(grad.shape[1]):
          grad[:,j] = dh[j]*grad_a[j]*self.last_x
        return grad

    def grad_b(self, dh):
        return dh * self.a.grad()*1

## 심층신경망 구현

In [11]:
class DNN:
  def __init__(self, hidden_depth, num_neuron, input, output, activation = Sigmoid):
    def init_var(i, o):
      return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,))
    
    self.sequence = []

    W, b = init_var(input, num_neuron)
    self.sequence.append(Neuron(W,b,activation))

    for _ in range(hidden_depth):
      W,b = init_var(num_neuron, num_neuron)
      self.sequence.append(Neuron(W,b,activation))

    W , b = init_var(num_neuron, output)
    self.sequence.append(Neuron(W,b,activation))

  def __call__(self, x):
    for layer in self.sequence:
      x = layer(x)
    return x

  def calc_gradient(self,loss_obj):
    loss_obj.dh = loss_obj.grad()
    self.sequence.append(loss_obj)

    #back propagation loop
    for i in range(len(self.sequence) -1 , 0 , -1):
      l1 = self.sequence[i]
      l0 = self.sequence[i-1]

      l0.dh = _m(l0.grad(), l1.dh)
      l0.dw = l0.grad_W(l1.dh)
      l0.db = l0.grad_b(l1.dh)
    self.sequence.remove(loss_obj)

## 경사하강 학습법

In [18]:
def gradient_descent(network, x, y, loss_obj, alpha = 0.01):
  loss = loss_obj(network(x), y)   #Forward inference
  network.calc_gradient(loss_obj)  #Back propagation
  for layer in network.sequence:   #모든 layer에 대해
    layer.W += -alpha * layer.dw
    layer.b += -alpha * layer.db
  return loss
    

## 동작 테스트

In [21]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth = 5, num_neuron = 32, input = 10, output = 2, activation= Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
  loss = gradient_descent(dnn, x, y, loss_obj, alpha = 0.01)
  print(f'Epoch {epoch} : Test loss {loss}')
print(f'{time.time() - t} seconds')

Epoch 0 : Test loss 0.19741929448160567
Epoch 1 : Test loss 0.19521024820223132
Epoch 2 : Test loss 0.19302793093961024
Epoch 3 : Test loss 0.19087249522854663
Epoch 4 : Test loss 0.18874406442880776
Epoch 5 : Test loss 0.18664273355262936
Epoch 6 : Test loss 0.18456857014272005
Epoch 7 : Test loss 0.18252161519396337
Epoch 8 : Test loss 0.1805018841121689
Epoch 9 : Test loss 0.17850936770341883
Epoch 10 : Test loss 0.1765440331877927
Epoch 11 : Test loss 0.17460582523152168
Epoch 12 : Test loss 0.17269466699192296
Epoch 13 : Test loss 0.17081046116978263
Epoch 14 : Test loss 0.16895309106419518
Epoch 15 : Test loss 0.16712242162521557
Epoch 16 : Test loss 0.16531830050003588
Epoch 17 : Test loss 0.16354055906876172
Epoch 18 : Test loss 0.1617890134662194
Epoch 19 : Test loss 0.1600634655865823
Epoch 20 : Test loss 0.158363704067953
Epoch 21 : Test loss 0.15668950525437478
Epoch 22 : Test loss 0.15504063413307434
Epoch 23 : Test loss 0.15341684524504842
Epoch 24 : Test loss 0.151817883